In [1]:
import pandas as pd
import numpy as np
import keras
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
import random
import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

account_path = 'kddi_r_account_full_20180410.tsv.gz'
order_path = 'kddi_r_pay_order.tsv.gz'
deal_path = 'kddi_r_deal.tsv_20180328.gz'
deal_profile_path = 'kddi_r_deal_profile_20180427.tsv.gz'
work_path = '/home/mmde-lab/s3/59e875d818c7b/'
deal_profile_path2 = 'kddi_r_deal_profile_20171222.tsv.gz'
#df_account = pd.read_csv(work_path + account_path, sep = '\t', compression='gzip')
df_order = pd.read_csv(work_path + order_path, sep = '\t', compression='gzip')
#df_deal = pd.read_csv(work_path + deal_path, sep = '\t', compression='gzip')
#df_dealprofile = pd.read_csv(work_path + deal_profile_path, sep = '\t', compression='gzip')
#df_dealprofile2 = pd.read_csv(work_path + deal_profile_path2, sep = '\t', compression='gzip')

Using TensorFlow backend.


In [2]:
df_order = df_order[['account_id', 'deal_id', 'ins_tm']]
df_order['rating'] = 1
#df_order.deal_id = df_order.deal_id.map(lambda x: str(x))
#df_order.account_id = df_order.account_id.map(lambda x: str(x))

In [3]:
len(df_order.account_id.value_counts())

341409

In [3]:
df_order['ins_tm'] = pd.to_datetime(df_order['ins_tm'],format='%Y-%m-%d %H:%M:%S')
df_order['date'] = df_order.ins_tm.dt.date
train_order = df_order.loc[df_order.date <= datetime.date(2017,9,14)]
valid_order = df_order.loc[df_order.date >= datetime.date(2017,9,15)]
valid_order = valid_order.loc[df_order.date <= datetime.date(2017,9,23)]

In [4]:
test_order = df_order.loc[df_order.date >= datetime.date(2017,9,24)]

In [6]:
train_order.shape, valid_order.shape, test_order.shape

((1582936, 5), (65564, 5), (34201, 5))

In [7]:
item_list = list(train_order.deal_id.unique())
user_list = list(train_order.account_id.unique())

In [8]:
len(item_list), len(user_list)

(64762, 328756)

In [9]:
(train_order.account_id.value_counts() >= 2).sum() / (train_order.account_id.value_counts() >= 1).sum() *100

60.28148535692124

# ----------------------------------------------------------------------------------
## user central negative sampling

In [15]:
item_list = list(train_order.deal_id.unique())
user_list = list(train_order.account_id.unique())

num_epo = 0
Negative_sample = []
for user in user_list:
    bought_list = list(train_order[train_order.account_id == user].deal_id.values)
    negative_list = list(set(item_list)^set(bought_list))
    some = random.sample(negative_list, len(bought_list))
    for i in range(len(bought_list)):
        Negative_sample.append([user, some[i], 0])
    if num_epo%10000 == 0:
        print(str(num_epo) + '/' + str(len(user_list)) + 'finished!')
    num_epo += 1
df_Ns = pd.DataFrame(Negative_sample, columns=['account_id', 'deal_id', 'rating'])
train_order = train_order.append(df_Ns, ignore_index=True)
train_order.to_csv('train_order_add_negasmpl.gz', compression = 'gzip')

0/328756finished!
10000/328756finished!
20000/328756finished!
30000/328756finished!
40000/328756finished!
50000/328756finished!
60000/328756finished!
70000/328756finished!
80000/328756finished!
90000/328756finished!
100000/328756finished!
110000/328756finished!
120000/328756finished!
130000/328756finished!
140000/328756finished!
150000/328756finished!
160000/328756finished!
170000/328756finished!
180000/328756finished!
190000/328756finished!
200000/328756finished!
210000/328756finished!
220000/328756finished!
230000/328756finished!
240000/328756finished!
250000/328756finished!
260000/328756finished!
270000/328756finished!
280000/328756finished!
290000/328756finished!
300000/328756finished!
310000/328756finished!
320000/328756finished!


In [15]:
item_list = list(set(valid_order.deal_id.unique())&set(train_order.deal_id.unique()))
user_list = list(set(valid_order.account_id.unique())&set(train_order.account_id.unique()))

num_epo = 0
Negative_sample = []
for user in user_list:
    bought_list = list(valid_order[valid_order.account_id == user].deal_id.values)
    negative_list = list(set(item_list)-set(bought_list))
    some = random.sample(negative_list, len(set(bought_list)&set(item_list)))
    for i in range(len(set(bought_list)&set(item_list))):
        Negative_sample.append([user, some[i], 0])
    if num_epo%10000 == 0:
        print(str(num_epo) + '/' + str(len(user_list)) + 'finished!')
    num_epo += 1
df_Ns = pd.DataFrame(Negative_sample, columns=['account_id', 'deal_id', 'rating'])
valid_order = valid_order.append(df_Ns, ignore_index=True)
valid_order.to_csv('valid_order_add_negasmpl.gz', compression = 'gzip')

0/33803finished!
10000/33803finished!
20000/33803finished!
30000/33803finished!


In [5]:
item_list = list(set(test_order.deal_id.unique())&set(train_order.deal_id.unique()))
user_list = list(set(test_order.account_id.unique())&set(train_order.account_id.unique()))

num_epo = 0
Negative_sample = []
for user in user_list:
    bought_list = list(test_order[test_order.account_id == user].deal_id.values)
    negative_list = list(set(item_list)-set(bought_list))
    some = random.sample(negative_list, len(set(bought_list)&set(item_list)))
    for i in range(len(set(bought_list)&set(item_list))):
        Negative_sample.append([user, some[i], 0])
    if num_epo%10000 == 0:
        print(str(num_epo) + '/' + str(len(user_list)) + 'finished!')
    num_epo += 1
df_Ns = pd.DataFrame(Negative_sample, columns=['account_id', 'deal_id', 'rating'])
test_order = test_order.append(df_Ns, ignore_index=True)
test_order.to_csv('test_order_add_negasmpl.gz', compression = 'gzip')

0/20044finished!
10000/20044finished!
20000/20044finished!


In [16]:
valid_order.rating.value_counts()

1    65564
0    33715
Name: rating, dtype: int64

# ----------------------------------------------------------------------------------
## product central negative sampling (only4 test set&valid set)

In [7]:
item_list = list(train_order.deal_id.unique())
user_list = list(train_order.account_id.unique())

num_epo = 0
Negative_sample = []
for item in item_list:
    bought_user = list(train_order[train_order.deal_id == item].account_id.values)
    negative_list = list(set(user_list)-set(bought_user))
    some = random.sample(negative_list, len(bought_user))
    for i in range(len(bought_user)):
        Negative_sample.append([some[i], item, 0])
    if num_epo%10000 == 0:
        print(str(num_epo) + '/' + str(len(item_list)) + 'finished!')
    num_epo += 1
df_Ns = pd.DataFrame(Negative_sample, columns=['account_id', 'deal_id', 'rating'])
train_order = train_order.append(df_Ns, ignore_index=True)
train_order.to_csv('train_order_add_negasmpl_dealcnt.gz', compression = 'gzip')

0/64762finished!
10000/64762finished!
20000/64762finished!
30000/64762finished!
40000/64762finished!
50000/64762finished!
60000/64762finished!


In [8]:
item_list = list(set(valid_order.deal_id.unique())&set(train_order.deal_id.unique()))
user_list = list(set(valid_order.account_id.unique())&set(train_order.account_id.unique()))

num_epo = 0
Negative_sample = []
for item in item_list:
    bought_user = list(valid_order[valid_order.deal_id == item].account_id.values)
    negative_list = list(set(user_list)-set(bought_user))
    some = random.sample(negative_list, len(set(bought_user)&set(user_list)))
    for i in range(len(set(bought_user)&set(user_list))):
        Negative_sample.append([some[i], item, 0])
    if num_epo%10000 == 0:
        print(str(num_epo) + '/' + str(len(item_list)) + 'finished!')
    num_epo += 1
df_Ns = pd.DataFrame(Negative_sample, columns=['account_id', 'deal_id', 'rating'])
valid_order = valid_order.append(df_Ns, ignore_index=True)
valid_order.to_csv('valid_order_add_negasmpl_dealcnt.gz', compression = 'gzip')

0/1284finished!


In [9]:
valid_order.rating.value_counts()

1    65564
0    33715
Name: rating, dtype: int64

In [10]:
item_list = list(set(test_order.deal_id.unique())&set(train_order.deal_id.unique()))
user_list = list(set(test_order.account_id.unique())&set(train_order.account_id.unique()))

num_epo = 0
Negative_sample = []
for item in item_list:
    bought_user = list(test_order[test_order.deal_id == item].account_id.values)
    negative_list = list(set(user_list)-set(bought_user))
    some = random.sample(negative_list, len(set(bought_user)&set(user_list)))
    for i in range(len(set(bought_user)&set(user_list))):
        Negative_sample.append([some[i], item, 0])
    if num_epo%10000 == 0:
        print(str(num_epo) + '/' + str(len(item_list)) + 'finished!')
    num_epo += 1
df_Ns = pd.DataFrame(Negative_sample, columns=['account_id', 'deal_id', 'rating'])
test_order = test_order.append(df_Ns, ignore_index=True)
test_order.to_csv('test_order_add_negasmpl_dealcnt.gz', compression = 'gzip')

0/183finished!


In [11]:
test_order.rating.value_counts()

1    34201
0     3332
Name: rating, dtype: int64

## ----------------------------------------------------------------------------------------------------------
## single sampling product central

In [39]:
item_list = list(set(test_order.deal_id.unique())&set(train_order.deal_id.unique()))
user_list = list(set(test_order.account_id.unique())&set(train_order.account_id.unique()))

num_epo = 0
single_sample = []
for item in item_list:
    bought_user = list(test_order[test_order.deal_id == item].account_id.values)
    bought_list = list(set(user_list)&set(bought_user))
    negative_list = list(set(user_list)-set(bought_user))
    if len(bought_list) >= 1:
        positive_some = random.sample(bought_list, 1)
        negative_some = random.sample(negative_list, 1)

        single_sample.append([positive_some[0], item, 1])
        single_sample.append([negative_some[0], item, 0])
    if num_epo%50 == 0:
        print(str(num_epo) + '/' + str(len(item_list)) + 'finished!')
    num_epo += 1
df_single_sample = pd.DataFrame(single_sample, columns=['account_id', 'deal_id', 'rating'])

df_single_sample.to_csv('test_order_single_sample.gz', compression = 'gzip')

0/183finished!
50/183finished!
100/183finished!
150/183finished!


## ----------------------------------------------------------------------------------------------------------
## single sampling user central

In [18]:
item_list = list(set(test_order.deal_id.unique())&set(train_order.deal_id.unique()))
user_list = list(set(test_order.account_id.unique())&set(train_order.account_id.unique()))

num_epo = 0
single_sample_uc = []
for user in user_list:
    bought_list = list(test_order[test_order.account_id == user].deal_id.values)
    bought_list = list(set(item_list)&set(bought_list))
    negative_list = list(set(item_list)-set(bought_list))
    if len(bought_list) >= 1:
        positive_some = random.sample(bought_list, 1)
        negative_some = random.sample(negative_list, 1)
        single_sample_uc.append([user, positive_some[0], 1])
        single_sample_uc.append([user, negative_some[0], 0])
    if num_epo%1000 == 0:
        print(str(num_epo) + '/' + str(len(user_list)) + 'finished!')
    num_epo += 1
df_single_sample_uc = pd.DataFrame(single_sample_uc, columns=['account_id', 'deal_id', 'rating'])

df_single_sample_uc.to_csv('test_order_simple_sample_uc.gz', compression = 'gzip')

0/20044finished!
1000/20044finished!
2000/20044finished!
3000/20044finished!
4000/20044finished!
5000/20044finished!
6000/20044finished!
7000/20044finished!
8000/20044finished!
9000/20044finished!
10000/20044finished!
11000/20044finished!
12000/20044finished!
13000/20044finished!
14000/20044finished!
15000/20044finished!
16000/20044finished!
17000/20044finished!
18000/20044finished!
19000/20044finished!
20000/20044finished!


In [19]:
df_single_sample_uc.rating.value_counts()

1    3155
0    3155
Name: rating, dtype: int64

# Fully distribution

In [56]:
item_list = list(set(test_order.deal_id.unique())&set(train_order.deal_id.unique()))
user_list = list(set(test_order.account_id.unique())&set(train_order.account_id.unique()))

num_epo = 0
Negative_sample = []
for user in user_list:
    bought_list = list(test_order[test_order.account_id == user].deal_id.values)
    bought_train_list = list(train_order[train_order.account_id == user].deal_id.values)
    bought_valid_list = list(valid_order[valid_order.account_id == user].deal_id.values)
    bought_list = list(set(item_list)&(set(bought_list)|set(bought_train_list)|set(bought_valid_list)))
    negative_list = list(set(item_list)-set(bought_list))
    #some = random.sample(negative_list, len(set(bought_list)&set(item_list)))
    for i in range(len(negative_list)):
        Negative_sample.append([user, negative_list[i], 0])
    if num_epo%10000 == 0:
        print(str(num_epo) + '/' + str(len(user_list)) + 'finished!')
    num_epo += 1
df_Ns = pd.DataFrame(Negative_sample, columns=['account_id', 'deal_id', 'rating'])
test_order = test_order.append(df_Ns, ignore_index=True)
test_order.to_csv('test_order_fully_distribution.gz', compression = 'gzip')

0/20044finished!
10000/20044finished!
20000/20044finished!


In [57]:
test_order.rating.value_counts()

0    3659416
1      34201
Name: rating, dtype: int64

# simple RS

In [2]:
train_order = pd.read_csv('train_order_add_negasmpl.gz', compression='gzip')
valid_order = pd.read_csv('valid_order_add_negasmpl.gz', compression='gzip')
test_order = pd.read_csv('test_order_add_negasmpl.gz', compression='gzip')

In [3]:
train_order.shape, valid_order.shape, test_order.shape

((3165872, 6), (131128, 6), (68402, 6))

In [4]:
n_users, n_item = len(df_order.account_id.unique()), len(df_order.deal_id.unique())
n_latent_factors = 3

In [16]:
item_input = keras.layers.Input(shape=[1], name='Item')
item_embedding = keras.layers.Embedding(n_item + 1, n_latent_factors, name='Item-Embedding')(item_input)
item_vec = keras.layers.Flatten(name='FlattenItem')(item_embedding)

user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))

dot = keras.layers.dot([item_vec, user_vec], -1, name='DotProduct')
#prod = output = keras.layers.Dense(1, activation='sigmoid')(dot)
model = keras.Model([item_input, user_input], dot)
model.compile('adam', 'mse', metrics=['accuracy'])

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Item-Embedding (Embedding)      (None, 1, 3)         205377      Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 3)         1024230     User[0][0]                       
__________________________________________________________________________________________________
FlattenIte

In [18]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

#check_point = ModelCheckpoint('model_naive.hdf5', verbose=True, save_best_only = True)
#early_stop = EarlyStopping(patience=5, verbose=True)
model.fit([train_order.deal_id, train_order.account_id], train_order.rating, 
          validation_data=([valid_order.deal_id, valid_order.account_id], valid_order.rating),
          batch_size=128,
          epochs=20, 
          verbose=True,
         #callbacks=[early_stop, check_point]
         )

Train on 3165872 samples, validate on 131128 samples
Epoch 1/20


InvalidArgumentError: indices[0,0] = 401554 is not in [0, 341410)
	 [[Node: User-Embedding_3/embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@training_3/Adam/Assign_5"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](User-Embedding_3/embeddings/read, User-Embedding_3/Cast, User-Embedding_3/embedding_lookup/axis)]]

In [22]:
model.load_weights('model.hdf5')

In [23]:
pred_result = model.predict([test_order.account_id, test_order.deal_id])

In [24]:
right_num = 0
for i in range(len(pred_result)):
    if int(pred_result[i][0] >= 0.5) == test_order.rating.values[i]:
        right_num += 1
print('accuracy: {}'.format((right_num / len(pred_result))))

accuracy: 0.5329522528581035


# result  n_latent_factor  optimizer  loss  acc
## 3  adam  mse  0.7603412362832463
## 3  SGD    mse  acc: 0.50
## 3  adam  crossEntropy 0.7822672423270864 (sigmoid(pred)
## 5  adam  crpssEntropy 0.7833131772948912 (sigmoid(pred)

n_latent_factor optimizer loss acc (train_set_embedding_only)
5  adam mse 0.533887898014678